In [1]:
import pandas as pd
import os
import numpy as np

In [9]:
INPUT_FOLDER = "C:\\Users\\sahil_pc\\Documents\\Flydubai\\flydubai structured format data"
BASE_SHEET = "Combine Base.xlsx"
BASE_SHEET_PKL = "base.pkl"

SSR_SHEET = "flydubai SSR Sep21_Feb22.xlsx"
SSR_SHEET_PKL = "ssr.pkl"

In [10]:
if os.path.exists(os.path.join(INPUT_FOLDER,BASE_SHEET_PKL)):
    df_base = pd.read_pickle(os.path.join(INPUT_FOLDER,BASE_SHEET_PKL))
else:
    df_base = pd.read_excel(os.path.join(INPUT_FOLDER,BASE_SHEET))
    df_base.to_pickle(os.path.join(INPUT_FOLDER,BASE_SHEET_PKL))
    
if os.path.exists(os.path.join(INPUT_FOLDER,SSR_SHEET_PKL)):
    df_ssr = pd.read_pickle(os.path.join(INPUT_FOLDER,SSR_SHEET_PKL))
else:
    df_ssr = pd.read_excel(os.path.join(INPUT_FOLDER,SSR_SHEET))
    df_ssr.to_pickle(os.path.join(INPUT_FOLDER,SSR_SHEET_PKL))

In [11]:
def clean_base_sheet(df):
    df["booking"] = np.where(df["pax_type_cde"] == "INF",1,df["booking"])
    a =pd.DataFrame(df.groupby(["confirmation_num","from_airport","to_airport","year(departure_dt)","month(departure_dt)","res_book_dt_lcl_key","departure_dt","journey_type"])[["booking","rev"]].sum().sort_values(by = "booking", ascending=False))
    a.to_csv("s.csv")
    b = pd.DataFrame(df.groupby(["confirmation_num","from_airport","to_airport","pax_type_cde"])[["booking"]].sum().sort_values(by = "booking", ascending=False))
    b.to_csv("b.csv")
    df = pd.read_csv("s.csv")
    df2 = pd.read_csv("b.csv")
    df3 = df2[df2["pax_type_cde"]=="ADT"]
    df3 = df3.drop("pax_type_cde", axis = 1)
    merged_df = pd.merge(df, df3, on=['confirmation_num', 'from_airport','to_airport'], how='left').fillna(0)
    df4 = df2[df2["pax_type_cde"]=="CHD"]
    df4 = df4.drop("pax_type_cde", axis = 1)
    merged_df2 = pd.merge(merged_df, df4, on=['confirmation_num', 'from_airport','to_airport'], how='left').fillna(0)
    merged_df2.rename(columns = {'booking_y':'ADT','booking':'CHILD' , 'booking_x' : 'pax' }, inplace = True)
    df5 = df2[df2["pax_type_cde"]=="INF"]
    df5 = df5.drop("pax_type_cde", axis = 1)
    merged_df3 = pd.merge(merged_df2, df5, on=['confirmation_num', 'from_airport','to_airport'], how='left').fillna(0)
    merged_df3["pax"] = np.where(merged_df3["booking"]>0 , merged_df3["pax"]-merged_df3["booking"],merged_df3["pax"])
    merged_df3.rename(columns = {'booking':'INF'}, inplace = True)
    merged_df3['id'] = df['confirmation_num'] + "-" + df['from_airport'] + "-" + df['to_airport']
    return merged_df3

In [18]:
def clean_ssr_sheet(df):
    df['id'] = df['pnr'] + "-" + df['from_od'] + "-" + df['to_od']
    return df_base

In [19]:
df_cleaned_base = clean_base_sheet(df_base)
df_cleaned_ssr = clean_ssr_sheet(df_ssr)

#df_cleaned_base = df_cleaned_base.drop_duplicates(subset=['id'], keep='first')  #ONLY DONE IF DISCREPANCY IN DATA

In [20]:
df_cleaned_base

,confirmation_num,from_airport,to_airport,year(departure_dt),month(departure_dt),res_book_dt_lcl_key,departure_dt,journey_type,pax,rev,ADT,CHILD,INF,id
0,RAALQX,DXB,GYD,2021,10,2021-10-27 00:00:00,2021-10-28 00:00:00,RT,7.0,3712.21,4.0,3.0,4.0,RAALQX-DXB-GYD
1,FP8S0J,DXB,TLV,2021,9,2021-09-14 00:00:00,2021-09-15 00:00:00,OW,8.0,11018.90,5.0,3.0,3.0,FP8S0J-DXB-TLV
2,QMI0J0,DXB,MHD,2021,12,2021-12-06 00:00:00,2021-12-07 00:00:00,OW,9.0,7961.39,7.0,2.0,2.0,QMI0J0-DXB-MHD
3,RAALQX,GYD,DXB,2021,10,2021-10-27 00:00:00,2021-10-31 00:00:00,RT,7.0,3315.00,4.0,3.0,4.0,RAALQX-GYD-DXB
4,XTBVKU,MSQ,DXB,2022,2,2022-01-25 00:00:00,2022-02-20 00:00:00,OW,9.0,6091.33,7.0,2.0,1.0,XTBVKU-MSQ-DXB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521546,EB3D1X,BEY,DXB,2021,11,2021-11-02 00:00:00,2021-11-04 00:00:00,OW,1.0,472.98,1.0,0.0,0.0,EB3D1X-BEY-DXB
521547,EB357V,DXB,DOH,2022,1,2022-01-02 00:00:00,2022-01-02 00:00:00,OW,1.0,707.39,1.0,0.0,0.0,EB357V-DXB-DOH
521548,EB314S,KBP,DXB,2021,12,2021-12-17 00:00:00,2021-12-27 00:00:00,RT,1.0,883.91,1.0,0.0,0.0,EB314S-KBP-DXB
521549,EB314S,DXB,KBP,2022,1,2021-12-17 00:00:00,2022-01-13 00:00:00,RT,1.0,818.72,1.0,0.0,0.0,EB314S-DXB-KBP


In [21]:
df_ssr

,pnr,leg_departure_date,od_flight_no,leg_flight_no,from_od,to_od,from_leg,to_leg,ssr_code,ssr_booked_date,...,ssr_booked_time_utc,ssr_booked_channel,ssr_paid_revenue,fare_brand_id,booking_channel_id,res_seg_status,ptc_id,charge_status,Fare Brand,id
0,M7FB94,20210901,1620/001,1620,EBB,DOH,EBB,DXB,SPST,20210817,...,2021-08-17 06:21:24.0,2,29.020000,6,2,5,1,1,Value,M7FB94-EBB-DOH
1,M7FB94,20210901,1620/001,1,EBB,DOH,DXB,DOH,NSST,20210817,...,2021-08-17 06:21:24.0,2,20.200000,6,2,5,1,1,Value,M7FB94-EBB-DOH
2,U5UPEV,20210901,406,406,BLR,DXB,BLR,DXB,NSST,20210831,...,2021-08-31 06:07:23.0,2,30.020000,6,2,5,1,1,Value,U5UPEV-BLR-DXB
3,UMIXTK,20210902,8198/1053,8198,DEL,TIA,DEL,DXB,BUPX,20210812,...,2021-08-12 10:20:49.0,2,107.186876,7,2,5,1,1,Flex,UMIXTK-DEL-TIA
4,YNNVG1,20210901,762,762,SJJ,DXB,SJJ,DXB,XLGR,20210811,...,2021-08-11 10:31:10.0,12,155.000000,7,2,5,1,1,Flex,YNNVG1-SJJ-DXB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259667,TAFRWU,20220228,002,2,DOH,DXB,DOH,DXB,MLIN,20220226,...,2022-02-26 18:32:45.0,12,35.300000,5,2,5,1,1,Lite,TAFRWU-DOH-DXB
259668,15JFZY,20220228,476,476,MAA,DXB,MAA,DXB,NSST,20211230,...,2021-12-30 17:33:33.0,2,29.990000,6,2,5,1,1,Value,15JFZY-MAA-DXB
259669,FC7PV3,20220228,438,438,AMD,DXB,AMD,DXB,XLGR,20220227,...,2022-02-27 18:51:14.0,2,138.010000,7,2,5,1,1,Flex,FC7PV3-AMD-DXB
259670,S15LTX,20220228,912/557,557,VKO,CMB,DXB,CMB,NSST,20220227,...,2022-02-27 14:05:06.0,2,30.000000,6,2,5,1,1,Value,S15LTX-VKO-CMB


In [14]:
#df_cleaned_base.to_excel("df_cleaned_base.xlsx")

In [22]:
unique_ids = df_cleaned_base['id'].tolist()
unique_ssr = set(df_ssr['ssr_code'].tolist())

In [23]:
df_final = df_cleaned_base.copy()
for i in unique_ssr:
    df_final[i] = 0

In [26]:
from datetime import datetime

start = datetime.now()

pnr_ssr_map = {k:{} for k in unique_ids}

for idx,uid in enumerate(unique_ids[:1000]):
    x = df_ssr[df_ssr['id'] == uid]
    for ssr in x['ssr_code'].to_list():
        pnr_ssr_map[uid][ssr] = pnr_ssr_map[uid].get(ssr,0) + 1
        
end = datetime.now()
print(end-start)

0:00:08.957790


In [27]:
for k,v in pnr_ssr_map.items():
    for ssr,cnt in v.items():
        df_final.loc[df_final['id']==k,ssr] = cnt 

In [41]:
# df_final.to_excel(os.path.join(INPUT_FOLDER,'final.xlsx'))

In [42]:
df_final.to_csv(os.path.join(INPUT_FOLDER,'final.csv'))

In [28]:
df_final

,confirmation_num,from_airport,to_airport,year(departure_dt),month(departure_dt),res_book_dt_lcl_key,departure_dt,journey_type,pax,rev,...,MLIN,BUPZ,BAGX,BUPX,JUBX,XLGR,NSST,SPEQ,BUPL,BAGB
0,RAALQX,DXB,GYD,2021,10,2021-10-27 00:00:00,2021-10-28 00:00:00,RT,7.0,3712.21,...,0,0,0,0,0,0,0,0,0,0
1,FP8S0J,DXB,TLV,2021,9,2021-09-14 00:00:00,2021-09-15 00:00:00,OW,8.0,11018.90,...,0,0,0,0,0,0,0,0,0,0
2,QMI0J0,DXB,MHD,2021,12,2021-12-06 00:00:00,2021-12-07 00:00:00,OW,9.0,7961.39,...,0,0,0,0,0,0,0,0,0,0
3,RAALQX,GYD,DXB,2021,10,2021-10-27 00:00:00,2021-10-31 00:00:00,RT,7.0,3315.00,...,0,0,0,0,0,0,0,0,0,0
4,XTBVKU,MSQ,DXB,2022,2,2022-01-25 00:00:00,2022-02-20 00:00:00,OW,9.0,6091.33,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521546,EB3D1X,BEY,DXB,2021,11,2021-11-02 00:00:00,2021-11-04 00:00:00,OW,1.0,472.98,...,0,0,0,0,0,0,0,0,0,0
521547,EB357V,DXB,DOH,2022,1,2022-01-02 00:00:00,2022-01-02 00:00:00,OW,1.0,707.39,...,0,0,0,0,0,0,0,0,0,0
521548,EB314S,KBP,DXB,2021,12,2021-12-17 00:00:00,2021-12-27 00:00:00,RT,1.0,883.91,...,0,0,0,0,0,0,0,0,0,0
521549,EB314S,DXB,KBP,2022,1,2021-12-17 00:00:00,2022-01-13 00:00:00,RT,1.0,818.72,...,0,0,0,0,0,0,0,0,0,0
